In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_type_code = df['Type_Code'].tolist()
list_part_key = df['Part_Key'].tolist()

print('总数量：' + str(len(list_part_key)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for type_code, part_key in zip(list_type_code, list_part_key):
    work.put_nowait((type_code, part_key))

def crawler():
    global df
    
    while not work.empty():
        type_code, part_key = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                if part_key == '':
                        df_temp = pd.DataFrame([{'status': 'ok',
                                                 'Part_Key': part_key,
                                                 'Vehicle': ''}])

                        status = 'ok'

                        break
                
                data = 'func=getbuyersguide&payload=' + quote('{"partData":{"listing_data_essential":{"parttype":"') + type_code + quote('","partkey":"') + part_key + quote('"}}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'},
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = html.xpath('//div[contains(@class, "buyersguide-nested")]/descendant::table[contains(@class, "nobmp")]/tr')

                vehicle = '\n'.join([' '.join(_.xpath('./td/text()')) for _ in list_vehicle])

                # = = = = = = = = = = = = = = =
           
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Key': part_key,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Key': part_key}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_key +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：351

677141  <->  [ok] - 剩余数量：336
677183  <->  [ok] - 剩余数量：335
677144  <->  [ok] - 剩余数量：334
59051  <->  [ok] - 剩余数量：333
4944726  <->  [ok] - 剩余数量：332
5402292  <->  [ok] - 剩余数量：331
59084  <->  [ok] - 剩余数量：330
677157  <->  [ok] - 剩余数量：329
59085  <->  [ok] - 剩余数量：328
59087  <->  [ok] - 剩余数量：327
59089  <->  [ok] - 剩余数量：326
59099  <->  [ok] - 剩余数量：325
59088  <->  [ok] - 剩余数量：324
357075  <->  [ok] - 剩余数量：323
59097  <->  [ok] - 剩余数量：322
59104  <->  [ok] - 剩余数量：321
59102  <->  [ok] - 剩余数量：320
59093  <->  [ok] - 剩余数量：319
59098  <->  [ok] - 剩余数量：318
4944588  <->  [ok] - 剩余数量：317
677188  <->  [ok] - 剩余数量：316
59071  <->  [ok] - 剩余数量：315
59113  <->  [ok] - 剩余数量：314
677186  <->  [ok] - 剩余数量：313
59130  <->  [ok] - 剩余数量：312
59115  <->  [ok] - 剩余数量：311
677191  <->  [ok] - 剩余数量：310
59127  <->  [ok] - 剩余数量：309
59119  <->  [ok] - 剩余数量：308
59117  <->  [ok] - 剩余数量：307
59126  <->  [ok] - 剩余数量：306
59133  <->  [ok] - 剩余数量：305
59134  <->  [ok] - 剩余数量：304
59137  <->  [ok] - 剩余数量：303
59123  <->  [ok] - 剩余数量：3

59584  <->  [ok] - 剩余数量：47
59577  <->  [ok] - 剩余数量：46
59560  <->  [ok] - 剩余数量：45
59583  <->  [ok] - 剩余数量：44
357114  <->  [ok] - 剩余数量：43
357111  <->  [ok] - 剩余数量：42
357112  <->  [ok] - 剩余数量：41
357115  <->  [ok] - 剩余数量：40
357113  <->  [ok] - 剩余数量：39
357104  <->  [ok] - 剩余数量：38
474143  <->  [ok] - 剩余数量：37
474127  <->  [ok] - 剩余数量：36
357120  <->  [ok] - 剩余数量：35
59591  <->  [ok] - 剩余数量：34
59555  <->  [ok] - 剩余数量：33
357110  <->  [ok] - 剩余数量：32
59593  <->  [ok] - 剩余数量：31
59592  <->  [ok] - 剩余数量：30
  <->  [ok] - 剩余数量：29
474129  <->  [ok] - 剩余数量：28
474142  <->  [ok] - 剩余数量：27
474138  <->  [ok] - 剩余数量：26
357122  <->  [ok] - 剩余数量：25
59575  <->  [ok] - 剩余数量：24
4944768  <->  [ok] - 剩余数量：23
4944534  <->  [ok] - 剩余数量：22
1042020  <->  [ok] - 剩余数量：21
474144  <->  [ok] - 剩余数量：20
474131  <->  [ok] - 剩余数量：19
474130  <->  [ok] - 剩余数量：18
1042019  <->  [ok] - 剩余数量：17
4944759  <->  [ok] - 剩余数量：16
59585  <->  [ok] - 剩余数量：15
474133  <->  [ok] - 剩余数量：14
4945041  <->  [ok] - 剩余数量：13
4944645  <->  [ok] - 剩余数量：12
4